In [1]:
! pip install transformers==4.28.0 datasets evaluate rouge_score sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
import pandas as pd

In [4]:
from datasets import Dataset

#### Load Data

In [5]:
with open ('/content/drive/MyDrive/AQA_CleanData/results_tfidf_questions_passages_df.pickle', 'rb') as f:
  results_tfidf = pickle.load(f)

In [6]:
inputs = []
target_outputs = []
for result in results_tfidf:
  input = result['question'] + ' </s> '
  target_output = result['answer']
  for ranked_passage in result['ranked_text'][:10]:
    input+= (ranked_passage +  ' </s> ')
  inputs.append(input)
  target_outputs.append(target_output)

In [7]:
data = {
    'inputs': inputs,
    'target_outputs': target_outputs
}
dataset = Dataset.from_dict(data)

#### Train Test Split

In [8]:
train_test = dataset.train_test_split(test_size=0.2, seed=42)

train_set = train_test["train"]
test_set = train_test["test"]

dataset = train_set.train_test_split(test_size=0.25, seed=42)

#### Tokenize for mT5

In [9]:
from transformers import AutoTokenizer

checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
def preprocess_function(sample):

    model_inputs = tokenizer(sample['inputs'], max_length=1024, truncation=True)
    labels = tokenizer(sample["target_outputs"], max_length=256, truncation=True, padding="max_length", return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6576 [00:00<?, ? examples/s]

Map:   0%|          | 0/2192 [00:00<?, ? examples/s]

#### Data Collator

In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

#### Model

In [13]:
import evaluate
rouge = evaluate.load("rouge")
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [14]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

#### Trainer

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="mT5-tfidf-10pass-all-questions-QA-22-06-2023",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/content/mT5-tfidf-10pass-all-questions-QA-22-06-2023 is already a clone of https://huggingface.co/natope/mT5-tfidf-10pass-all-questions-QA-22-06-2023. Make sure you pull the latest changes with `repo.git_pull()`.


#### Training

In [17]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.307400,2.308976,0.080200,0.006700,0.071100,0.071100,15.492200
2,2.716100,2.122734,0.080500,0.016600,0.066500,0.066400,13.497700
3,2.609900,2.105196,0.135000,0.029300,0.109100,0.109100,18.364100


TrainOutput(global_step=9864, training_loss=4.934820306658842, metrics={'train_runtime': 3189.505, 'train_samples_per_second': 6.185, 'train_steps_per_second': 3.093, 'total_flos': 2.086234780336128e+16, 'train_loss': 4.934820306658842, 'epoch': 3.0})

In [18]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.12G [00:00<?, ?B/s]

Upload file runs/Jun22_01-04-17_cac05d977ee0/events.out.tfevents.1687395862.cac05d977ee0.27051.0:   0%|       …

To https://huggingface.co/natope/mT5-tfidf-10pass-all-questions-QA-22-06-2023
   2792441..31ef1b9  main -> main

   2792441..31ef1b9  main -> main

To https://huggingface.co/natope/mT5-tfidf-10pass-all-questions-QA-22-06-2023
   31ef1b9..96b417f  main -> main

   31ef1b9..96b417f  main -> main



'https://huggingface.co/natope/mT5-tfidf-10pass-all-questions-QA-22-06-2023/commit/31ef1b95101859ba14770e34e4ca46031f145cf3'